In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%pylab inline

from datetime import datetime,timedelta
import json
import re

from collections import OrderedDict
from math import pi
import pytz
import time
import psycopg2
from sqlalchemy import create_engine

from scipy import stats
import csv

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, normalize
import seaborn.apionly as sns

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('balanced_data.csv')

/Users/amiee/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.shape

(700000, 27)

In [4]:
data.ix[0,:]

advertiser_app_store_id                                  1009442510
country_code                                                     CN
city                                                            NaN
campaign_id                                56fc3b248409c5677800005c
creative_id                                582efacc4156ab1024000034
device_language                                                  zh
device_make                                                   iPad3
device_model                                                      4
device_platform                                                 iOS
device_connection                                              wifi
device_os_version                                              10.2
device_screen_height                                           2048
device_screen_width                                            1536
device_volume                                                 0.375
event_id                                   58674

### Convert timestamps to date format

In [10]:
cols = ['time_of_last_delivery_this_campaign', 'time_of_last_delivery_this_creative', 'time_of_last_delivery_any_installed_app',
       'time_of_last_delivery_any_installed_app','time_of_last_vungle_delivery', 'time_of_this_impression','time_of_this_request']
for col in cols:
#     print col
    name = col + "_date"
    data[name] = data[col].apply(lambda x: pd.to_datetime(x).date())

time_of_last_delivery_this_campaign
time_of_last_delivery_this_creative
time_of_last_delivery_any_installed_app
time_of_last_delivery_any_installed_app
time_of_last_vungle_delivery
time_of_this_impression
time_of_this_request


### get continent info from time zone

In [11]:
def get_continent(x):
    if len(str(x).split('/')) > 1:
        return str(x).split('/')[0]
    else:
        return None

In [12]:
data['time_zone_continent'] = data['time_zone'].apply(lambda x: get_continent(x))

### standardize the format of device_language

In [14]:
data['device_language'] = data['device_language'].apply(lambda x: str(x).lower().split('-')[0])

### grouping insignificant data (sig level = 0.95)

In [15]:
def new_col_sig95(df, col):
    sig_level = 0.05
    df = df.copy()
    items = df[col].unique()
    for i in items:
        a = df[df[col] == i]['is_install']
        b = df[df[col] != i]['is_install']
        t, p = stats.ttest_ind(a, b, equal_var=False)
        if p >= sig_level:
            df[col][df[col] == i] = 'other'
    return df[col]

In [16]:
data.ix[0,:]

advertiser_app_store_id                                       1009442510
country_code                                                          CN
city                                                                 NaN
campaign_id                                     56fc3b248409c5677800005c
creative_id                                     582efacc4156ab1024000034
device_language                                                       zh
device_make                                                        iPad3
device_model                                                           4
device_platform                                                      iOS
device_connection                                                   wifi
device_os_version                                                   10.2
device_screen_height                                                2048
device_screen_width                                                 1536
device_volume                                      

In [17]:
cols = ['advertiser_app_store_id', 'country_code', 'device_language', 'device_platform',
       'device_connection', 'time_zone_continent']
for col in cols:
#     print col
    name = col + '_95'
    data[name] = new_col_sig95(data, col)

advertiser_app_store_id


/Users/amiee/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amiee/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


country_code
device_language
device_platform
device_connection
n_campaign_views
time_zone_continent


In [18]:
data

,advertiser_app_store_id,country_code,city,campaign_id,creative_id,device_language,device_make,device_model,device_platform,device_connection,...,time_of_last_vungle_delivery_date,time_of_this_impression_date,time_of_this_request_date,advertiser_app_store_id_95,country_code_95,device_language_95,device_platform_95,device_connection_95,n_campaign_views_95,time_zone_continent_95
0,1009442510,CN,NaN,56fc3b248409c5677800005c,582efacc4156ab1024000034,zh,iPad3,4,iOS,wifi,...,2016-12-31,2016-12-31,2016-12-31,1009442510,CN,zh,iOS,wifi,4,Asia
1,5743f032a5a36ff4300000a5,ID,NaN,581a0af506c6c9996c000073,58465ef52c19fc6503000081,id,LENOVO,Lenovo A6010,android,mobile,...,2016-12-29,2016-12-31,2016-12-29,5743f032a5a36ff4300000a5,ID,id,android,mobile,0,Asia
2,727296976,CA,NaN,554d597375fbbeb654000163,57767bf0ef4e66dc5f000088,en,iPad6,3,iOS,wifi,...,2016-12-29,2016-12-29,2016-12-29,727296976,CA,en,iOS,wifi,0,America
3,57a28ffbb019f8257c00021d,LA,NaN,580fe05d6000a03f2d00015f,584267696dea79c175000030,th,iPhone7,2,iOS,mobile,...,2016-12-30,2016-12-30,2016-12-30,other,LA,th,iOS,mobile,1,Asia
4,com.plarium.vikings,CZ,NaN,585435605c9661f31200003c,585aef35a76e13401e0038c7,cs,Archos,Archos 55 diamond Selfie,android,wifi,...,2016-12-29,2016-12-30,2016-12-30,com.plarium.vikings,CZ,cs,android,wifi,0,Europe
5,558bdb44bbed958866000191,CN,NaN,56e93be0eccafb7e3500006b,57bc42dc627003b41f00006b,zh,iPhone7,2,iOS,NaN,...,2016-12-22,2016-12-29,2016-12-29,558bdb44bbed958866000191,CN,zh,iOS,NaN,0,Asia
6,com.skout.android,BR,NaN,5260798dc76408c54f00000f,5600615480fbbf1028000184,pt,asus,ASUS_Z00VD,android,wifi,...,2016-12-29,2016-12-29,2016-12-29,com.skout.android,BR,pt,android,wifi,0,America
7,58534782688691e41e000011,US,NaN,5853f8293adda7a60c000024,5860c91d156bf31f650001b0,en,iPhone8,2,iOS,NaN,...,2016-12-29,2016-12-30,2016-12-30,58534782688691e41e000011,US,en,iOS,NaN,1,America
8,585215f054dbb89701000ba2,US,NaN,585247d492f308b72a00015a,5860bde8156bf31f65000198,en,samsung,SM-T357T,android,wifi,...,2016-12-30,2017-01-01,2016-12-30,585215f054dbb89701000ba2,US,en,android,wifi,8,America
9,937718942,US,NaN,582f9de0740cf4426b1d5338,5848d7a6b942517e03000113,en,iPad3,4,iOS,wifi,...,2016-12-30,2016-12-31,2016-12-30,937718942,US,en,iOS,wifi,2,America


In [23]:
data.to_csv('balanced_data_grouped.csv')